# Baseline Forecast using NIXTLA

### Loading Libraries

In [2]:
%cd ../..

/Users/joaquinromero/Desktop


In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Warnings
import warnings
import humanize

# IO & Requests
import time
import random
import requests
from io import StringIO

# StatsModels
import statsmodels.api as sm
from statsmodels.tsa.seasonal import MSTL , DecomposeResult

# OS
import os
import sys
import pickleshare
import missingno as msno
from itertools import cycle

# PyArrow
import pyarrow as pa

# FuncTools
from functools import partial

# Path & Notebook Optimizer
from pathlib import Path
import missingno as msno
from tqdm.auto import tqdm

# Scikit-Learn
from sklearn.metrics import mean_absolute_error

# IPython
from IPython.display import display, HTML

# NIXTLA
from statsforecast.core import StatsForecast
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    WindowAverage,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    HoltWinters,
    ETS,
    AutoETS,
    AutoARIMA,
    ARIMA,
    AutoTheta,
    DynamicTheta,
    DynamicOptimizedTheta,
    Theta,
    OptimizedTheta,
    TBATS,
    AutoTBATS,
    MSTL
)

# Forecast
from datasetsforecast.losses import *

# Custom Libraries
from src.utils import plotting_utils
from src.utils.ts_utils import forecast_bias


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
tqdm.pandas()

random.seed(42)

np.random.seed(42)

pio.templates.default = "plotly_white"

sys.path.append('/Users/joaquinromero/Desktop/MTSF')

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

warnings.filterwarnings("ignore", category=FutureWarning)

warnings.filterwarnings("ignore", message="'force_all_finite' was renamed to 'ensure_all_finite'")

In [ ]:
# This Makes it so that The Outputs of The Predict Methods have The ID as a Column 
# Instead of as The Index
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [ ]:
os.makedirs("imgs/chapter_04", exist_ok=True)

preprocessed = Path.home() / "Desktop" / "data" / "london_smart_meters" / "preprocessed"

In [ ]:
from itertools import cycle

def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
        
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
            yanchor='top',
            font=dict(size=20)
        ),
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title=dict(text=ylabel, font=dict(size=font_size)),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title=dict(text=xlabel, font=dict(size=font_size)),
            tickfont=dict(size=font_size),
        )
    )
    return fig

In [1]:
def plot_forecast(pred_df, forecast_columns, timestamp_col, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [c.replace("rgb", "rgba").replace(")", ", <alpha>)") for c in px.colors.qualitative.Dark2]
    act_color = colors[0]
    colors = cycle(colors[1:])
    dash_types = cycle(["dash", "dot", "dashdot"])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df[mask]['energy_consumption'],
                             mode='lines', line=dict(color=act_color.replace("<alpha>", "0.3")),
                             name='Actual Consumption'))
    
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df.loc[mask, col],
                                 mode='lines', line=dict(dash=next(dash_types), color=next(colors).replace("<alpha>", "1")),
                                 name=display_col))
    return fig

In [ ]:
# import plotly.graph_objects as go
# import plotly.express as px
# from itertools import cycle

# def plot_forecast(pred_df, forecast_columns, timestamp_col, forecast_display_names=None):
#     # Showed Validated Names 
#     if forecast_display_names is None:
#         forecast_display_names = forecast_columns
#     else:
#         if not isinstance(forecast_display_names, list):
#             forecast_display_names = list(forecast_display_names)
#         assert len(forecast_columns) == len(forecast_display_names), "Display names must match forecast columns"
    
#     # Build-Up Null Values Mask
#     mask = ~pred_df[forecast_columns[0]].isnull()

#     # Switching RGB to RGBA Modifibly Opacity 
#     def rgba(color, alpha):
#         return color.replace("rgb", "rgba").replace(")", f", {alpha})")
    
#     colors = cycle([rgba(c, "1.0") for c in px.colors.qualitative.Dark2[1:]])
#     act_color = rgba(px.colors.qualitative.Dark2[0], "0.3")
#     dash_types = cycle(["dash", "dot", "dashdot"])

#     # Figure Build Up 
#     fig = go.Figure()

#     # Real Line 
#     fig.add_trace(go.Scatter(
#         x=pred_df.loc[mask, timestamp_col],
#         y=pred_df.loc[mask, 'energy_consumption'],
#         mode='lines',
#         line=dict(color=act_color),
#         name='Actual Consumption'
#     ))

#     # Forecast Lines 
#     for col, display_name in zip(forecast_columns, forecast_display_names):
#         fig.add_trace(go.Scatter(
#             x=pred_df.loc[mask, timestamp_col],
#             y=pred_df.loc[mask, col],
#             mode='lines',
#             line=dict(dash=next(dash_types), color=next(colors)),
#             name=display_name
#         ))

#     return fig

In [ ]:
# Reading The Missing Value Imputed and Train Test Split Data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed.parquet")
    train_df = train_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed.parquet")
    val_df = val_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    test_df = pd.read_parquet(preprocessed/"selected_blocks_test_missing_imputed.parquet")
    test_df = test_df[['LCLid',"timestamp","energy_consumption","frequency"]]
except FileNotFoundError:
    print(f"Warning: File not found in {preprocessed}. Ensure you've run '01-Setting up Experiment Harness.ipynb' in Chapter 04 and that the file path is correct.")


In [ ]:
print("Min train_df Date: " , train_df.timestamp.min())
print("Max train_df Date: " , train_df.timestamp.max())
print("Min val_df Date: " , val_df.timestamp.min())
print("Max val_df Date: " , val_df.timestamp.max())
print("Min test_df Date: " , test_df.timestamp.min())
print("Max test_df Date: " , test_df.timestamp.max())

In [ ]:
train_df = train_df[train_df.timestamp >'2012-01-01']
print("Min train_df Date: " , train_df.timestamp.min())

In [ ]:
# Picking a Single Time Series from The Dataset for Illustration
freq = train_df.iloc[0]['frequency']

ts_train = train_df.loc[train_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_val = val_df.loc[val_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_test = test_df.loc[test_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]

### Baseline Forecasts

In [ ]:
pred_df = pd.concat([ts_train, ts_val])

In [ ]:
def evaluate_performance(ts_train, ts_test, models, metrics, freq, level, id_col, time_col, target_col, h, metric_df=None):
    if metric_df is None:
        metric_df = pd.DataFrame()  # Initialize an empty DataFrame if not provided

    results = ts_test.copy()

    # Timing dictionary to store train and predict durations
    timing = {}

    for model in models:
        model_name = model.__class__.__name__
        evaluation = {}  # Reset the evaluation dictionary for each model

        # Start the timer for fitting and prediction
        start_time = time.time()

        # Instantiate StatsForecast class
        sf = StatsForecast(
            models=[model],
            freq=freq,
            n_jobs=-1,
            fallback_model=Naive()
        )

        # Efficiently predict without storing memory
        y_pred = sf.forecast(
            h=h,
            df=ts_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        )

        # Calculating the duration
        duration = time.time() - start_time
        timing[model_name] = duration

        # Merge prediction results to the original dataframe
        results = results.merge(y_pred, how='left', on=[id_col, time_col])

        ids = ts_train[id_col].unique()
        # Calculate metrics
        for id in ids:
            temp_results = results[results[id_col] == id]
            temp_train = ts_train[ts_train[id_col] == id]
            for metric in metrics:
                metric_name = metric.__name__
                if metric_name == 'mase':
                    evaluation[metric_name] = metric(temp_results[target_col].values,
                                                    temp_results[model_name].values,
                                                    temp_train[target_col].values, seasonality=48)
                else:
                    evaluation[metric_name] = metric(temp_results[target_col].values, temp_results[model_name].values)
            evaluation[id_col] = id
            evaluation['Time Elapsed'] = timing[model_name]

            # Prepare and append this model's results to metric_df
            temp_df = pd.DataFrame(evaluation, index=[0])
            temp_df['Model'] = model_name
            metric_df = pd.concat([metric_df, temp_df], ignore_index=True)

    return results, metric_df

### Naïve Forecast

In [ ]:
metrics = pd.DataFrame()

results, metrics = evaluate_performance(
    ts_train=ts_train, 
    ts_test=ts_val, 
    models=[Naive()], 
    metrics=[mase, mae, mse, rmse, smape, forecast_bias], 
    freq=freq,
    level=[],  # Ensure this is correct or adjust as necessary
    id_col='LCLid',
    time_col='timestamp',
    target_col='energy_consumption',
    h=len(ts_val),
    metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
)

In [ ]:
metrics

In [ ]:
model_name = ['Naive']
model_display_name = ['Naive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/naive.png")
fig.show()

### Seasonal Naïve Forecast

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train=ts_train, 
        ts_test=ts_val, 
        models = [SeasonalNaive(season_length=48*7)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
results.head()

In [ ]:
model_name = ['SeasonalNaive']
model_display_name = ['SeasonalNaive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/seasonal_naive.png")
fig.show()

### Moving Average Forecast

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [WindowAverage(window_size = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['WindowAverage']
model_display_name = ['WindowAverage']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/window_average.png")
fig.show()

### Exponential Smoothing Forecast

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ HoltWinters(error_type = 'A', season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['HoltWinters']
model_display_name = ['HoltWinters']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ AutoETS(model = 'AAA',season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
results.head()

In [ ]:
metrics

In [ ]:
model_name = ['AutoETS']
model_display_name = ['AutoETS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

In [ ]:
### ARIMA

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ ARIMA(order = (2,1,1), seasonal_order = (1,1,1), season_length = 48)],
        #models = [ ARIMA(order = (0,1,2), seasonal_order = (0,0,2), season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['ARIMA']
model_display_name = ['ARIMA']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ARIMA.png")
fig.show()

### Sample Auto ARIMA

In [ ]:
# sf = StatsForecast(
#     models=[AutoARIMA( max_p = 2, max_d=1, max_q = 2, max_P=2, max_D = 1, max_Q = 2, 
#                       start_p = 1, start_q = 1, start_P = 1, start_Q = 1, stepwise = True, season_length=48)],
#     freq=freq,
#     n_jobs=-1,
#     fallback_model = Naive()
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [ ]:
# {'coef': {'ma1': -0.3800676288610244,
#   'ma2': -0.25121322485842584,
#   'sma1': 0.1300139905884248,
#   'sma2': 0.1045851891918472},
#  'sigma2': 0.05918752833035802,
#  'var_coef': array([[ 7.04752468e-07,  9.11329831e-07,  2.55693473e-07,
#          -1.87114423e-06],
#         [ 9.11329831e-07,  7.41876851e-06, -3.18464778e-07,
#          -8.00782484e-06],
#         [ 2.55693473e-07, -3.18464778e-07,  5.50841803e-07,
#          -4.88097177e-07],
#         [-1.87114423e-06, -8.00782484e-06, -4.88097177e-07,
#           1.03626760e-05]]),
#  'mask': array([ True,  True,  True,  True]),
#  'loglik': -188.2911979664168,
#  'aic': 386.5823959328336,
#  'arma': (0, 2, 0, 2, 48, 1, 0),
#  'residuals': array([ 0.000368  ,  0.01585512, -0.19137661, ..., -0.3180852 ,
#         -0.35435519, -0.1594367 ]),
#  'code': 2,
#  'n_cond': 0,
#  'nobs': 35087,
# ...
#  'bic': 428.9103257852313,
#  'aicc': 386.58410626036135,
#  'ic': None,
#  'xreg': None,
#  'x': array([0.368, 0.386, 0.17 , ..., 0.147, 0.111, 0.09 ], dtype=float32),
#  'lambda': None}

### Theta

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ Theta(season_length =48, decomposition_type = 'additive' )], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
model_name = ['Theta']
model_display_name = ['Theta']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_theta.png")
fig.show()

In [ ]:
metrics

### TBATS

In [ ]:
# sf = StatsForecast(
#     models=[TBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
#     freq=freq,
#     n_jobs=2
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [ ]:
# sf = StatsForecast(
#     models=[AutoTBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
#     freq=freq,
#     n_jobs=2
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ TBATS(seasonal_periods = [48])], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['TBATS']
model_display_name = ['TBATS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

### MSTL

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ MSTL(season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
model_name = ['MSTL']
model_display_name = ['MSTL']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

In [ ]:
metric_styled = metrics.reset_index(drop=True).style.format({
            "mae": "{:.3f}", 
            "mse": "{:.3f}", 
            "mase": "{:.3f}", 
            "rmse": "{:.3f}", 
            "smape": "{:.3f}",            
            "forecast_bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["mae","mse","mase","rmse","smape","Time Elapsed"])
display(metric_styled)

## Validation Set

### Running Baseline Forecast for All Consumers

In [ ]:
ids = list(train_df.LCLid.unique()[:500]) # slicing dataframe for the sake of time.  May want to consider setting this low if working on a slower machine

train_df = train_df[train_df.LCLid.isin(ids)]
val_df = val_df[val_df.LCLid.isin(ids)]
test_df = test_df[test_df.LCLid.isin(ids)]

print("Length of validation Data: ", len(val_df[val_df.LCLid =='MAC000948'].LCLid))

In [ ]:
### Forecasting for Validation Period for TBATS and AutoETS

In [ ]:
validation_models =  [AutoETS(model = 'AAA',season_length = 48), 
                      TBATS(seasonal_periods  = 48)]

# validation_models = [Naive()]
validation_models_names = [model.__class__.__name__ for model in validation_models]
metric_df = pd.DataFrame([])
h_val = 1488

In [ ]:
aggval_metrics = pd.DataFrame()

baseline_val_pred_df, aggval_metrics = (
    evaluate_performance(
        train_df[["LCLid","timestamp","energy_consumption"]], 
        val_df[["LCLid","timestamp","energy_consumption"]], 
        models =validation_models, 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = h_val,
        metric_df = aggval_metrics
        )
)

#### Evaluation of Baseline Forecast

In [ ]:
aggval_metrics.head()

In [ ]:
baseline_val_pred_df[baseline_val_pred_df.LCLid =='MAC000173']

In [ ]:
aggval_metrics[aggval_metrics.Model =='TBATS'].sort_values(by='mase', ascending = False)

In [ ]:
autoets_val_metric_df = aggval_metrics[aggval_metrics.Model =='AutoETS']
overall_metrics_val_autoets = {
    "Algorithm": "AutoETS",
    "MAE": mae(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "MSE": mse(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "meanMASE": autoets_val_metric_df.mase.mean(),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values)
}
overall_metrics_val_autoets

In [ ]:
autoets_val_metric_df = aggval_metrics[aggval_metrics.Model =='AutoETS']
overall_metrics_val_autoets = {
    "Algorithm": "AutoETS",
    "MAE": mae(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "MSE": mse(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "meanMASE": autoets_val_metric_df.mase.mean(),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values)
}
overall_metrics_val_autoets

In [ ]:
baseline_val_metrics_df = pd.DataFrame([overall_metrics_val_autoets, overall_metrics_val_tbats])

display(baseline_val_metrics_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]))

In [ ]:
baseline_val_metrics_df.head()

In [ ]:
fig = px.histogram(aggval_metrics, 
                   x="mase", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mase", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,6])
# fig.write_image("imgs/chapter_4/mase_dist.png")
fig.show()

In [ ]:
fig = px.histogram(aggval_metrics, 
                   x="forecast_bias", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=250, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="forecast_bias", ylabel="Probability Density", title="Distribution of BIAS in the dataset")
fig.update_layout(xaxis_range=[-200,200])
# fig.write_image("imgs/chapter_4/bias_dist.png")
fig.show()

In [ ]:
fig = px.histogram(aggval_metrics, 
                   x="mae", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mae", ylabel="Probability Density", title="Distribution of MAE in the dataset")
fig.update_layout(xaxis_range=[0,1])
# fig.write_image("imgs/chapter_4/mae_dist.png")
fig.show()

In [ ]:
fig = px.histogram(aggval_metrics, 
                   x="mse", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mse", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,1.5])
# fig.write_image("imgs/chapter_4/mse_dist.png")
fig.show()

#### Saving The Baseline Forecasts & Metrics

In [ ]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)

output = Path("data/london_smart_meters/output")

baseline_val_pred_df.to_pickle(output/"baseline_val_prediction_df.pkl")
baseline_val_metrics_df.to_pickle(output/"baseline_val_metrics_df.pkl")
aggval_metrics.to_pickle(output/"baseline_val_aggregate_metrics.pkl")

## Test Set

### Forecasting for Test Period

In [ ]:
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse as rmse_local
from utilsforecast.losses import mae as mae_local
from utilsforecast.losses import mse as mse_local
from utilsforecast.losses import mase as mase_local

from functools import partial

In [ ]:
_train_df = pd.concat([train_df, val_df])
print("Length of training Data: ", len(test_df[test_df.LCLid =='MAC000948'].LCLid))
test_df.tail()

In [ ]:
test_models =  [ AutoETS(model = 'AAA',season_length = 48), TBATS(seasonal_periods  = 48)]
h_test = 1296

In [ ]:
sf = StatsForecast(
    models=test_models,
    freq=freq,
    n_jobs=-1,
    fallback_model= SeasonalNaive(season_length=48)
)

# sf.fit( df = _train_df[['timestamp', 'LCLid', 'energy_consumption']] ,    
#         id_col = 'LCLid',
#         time_col = 'timestamp',
#         target_col = 'energy_consumption',
#         )

# baseline_test_pred_df = sf.predict( h =1296 )

# Memory efficient predictions
baseline_test_pred_df = sf.forecast(df=_train_df, 
                                        h=h_test, 
                                        level=[],   
                                        id_col = 'LCLid',
                                        time_col = 'timestamp',
                                        target_col = 'energy_consumption',
        )
baseline_test_pred_df = pd.merge(baseline_test_pred_df, test_df[['timestamp', 'LCLid', 'energy_consumption']], on = ['LCLid','timestamp'], how = 'left')


In [ ]:
baseline_test_pred_df.head()

In [ ]:
fcst_mase = partial(mase_local, seasonality=48)

baseline_test_metrics_df = evaluate(baseline_test_pred_df, 
        metrics=[rmse_local, mae_local, mse_local,fcst_mase],  
        train_df = _train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption'
        )

In [ ]:
baseline_test_metrics_df.head()

In [ ]:
cols_to_exclude  = ['LCLid', 'metric']
test_columns = [col for col in baseline_test_metrics_df.columns if col not in cols_to_exclude]
test_columns

In [ ]:
baseline_test_metrics_df = pd.melt(baseline_test_metrics_df, id_vars = ['LCLid','metric'],value_vars= test_columns, var_name='Algorithm' , value_name='value')
baseline_test_metrics_df = baseline_test_metrics_df.pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'value').reset_index()
baseline_test_metrics_df.head()

In [ ]:
validation_models_names

In [ ]:
baseline_test_metrics_df.head()

In [ ]:
from datasetsforecast.losses import *

agg_test_metrics = []  # Initialize an empty list to store the metrics dictionaries

for model in validation_models_names:
    actual_series = baseline_test_pred_df['energy_consumption'].values
    pred_series = baseline_test_pred_df[model].values
    
    # Create a dictionary for the current model's metrics
    agg_test_metrics1 = {
        "Algorithm": model,
        "MAE": mae(actual_series, pred_series),
        "MSE": mse(actual_series, pred_series),
        "meanMASE": baseline_test_metrics_df[baseline_test_metrics_df.Algorithm == model].mase.mean(),
        "Forecast Bias": forecast_bias(actual_series, pred_series),
    }
    
    # Append the dictionary to the list
    agg_test_metrics.append(agg_test_metrics1)
agg_test_metrics_df = pd.DataFrame(agg_test_metrics)
agg_test_metrics_df